In [1]:
#Import modules
import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import patches, text, patheffects
import cv2
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

In [54]:
def group_DBSCAN(df):
    x = df[["c_x", "c_y"]].to_numpy()
    #hcluster = []
    #comp = int(len(df)) / len(((df["labeler_id"].unique())))
    cluster = DBSCAN(eps=15, min_samples=5).fit(x)
    labels = cluster.labels_
    df["cluster_id"] = labels
    #n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    #n_noise_ = list(labels).count(-1)
    #decision = comp / int(n_clusters_)
    #if decision > 0.9:
    #    df["cluster_id"] = labels
    return labels
    #else:
    #    hcluster.append(df["filename"])
    #    return print("Number of clusters differs from expected by >10%, please verify")

In [3]:
#JSON file with all image labels
labelpath = r"/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/CV4Ecology/Prototyping/Data/Labels/coco/20220227_dgc.json"
with open(labelpath) as f:
  #df = pd.read_json(f)
  cocozoo = pd.read_json(f) #delete this and use above instead once ready to run with all images

#Directory of images-- remember to change this to the full directory when the script is ready
imgpath = r"/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/CV4Ecology/Prototyping/Data/Imagery/SB597/scratch/test"
imglist = os.listdir(imgpath)

In [5]:
#Creating a multi-image subset to work with (delete this for full version)
df = cocozoo[cocozoo["filename"].isin(imglist)]

In [41]:
#CREATE NEW COLUMNS FOR INPUT INTO DBSCAN

#Derive bounding box centers
bboxes = df["bbox"]
c_x = []
c_y = []
x = []
y = []
w = []
h = []
centers = []
for coord in bboxes:
    center = (coord[0]+(coord[2]/2), coord[1]+(coord[3]/2))
    c_x.append(center[0])
    c_y.append(center[1])
    x.append(coord[0])
    y.append(coord[1])
    w.append(coord[2])
    h.append(coord[3])
    centers.append(center)
#Make these centers into a coordinate format
coords = []
for row in centers:
    coord = list(row)
    coords.append(coord)
#Append new columns to dataframe for manipulating later
df["c_x"] = c_x
df["c_y"] = c_y
df['x'] = x
df['y'] = y
df['w'] = w
df['h'] = h

/var/folders/21/q9_tvtjj53n6mfprd_174cz00000gn/T/ipykernel_99034/3272824413.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["c_x"] = c_x
/var/folders/21/q9_tvtjj53n6mfprd_174cz00000gn/T/ipykernel_99034/3272824413.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["c_y"] = c_y
/var/folders/21/q9_tvtjj53n6mfprd_174cz00000gn/T/ipykernel_99034/3272824413.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [39]:
df.head()

,annotation_id,bbox,area,category_id,category,image_id,filename,labeler_id,centers,x,y,w,h,c_x,c_y
15013,15014,"[314.8125, 131, 91, 43]",3913.0,3,Duck,2536,BDA_18a4_20181106_2_00295_01_04.png,437,360.3125,314.8125,131.0,91.0,43.0,360.3125,152.5
15014,15015,"[266.8125, 7, 55, 59]",3245.0,3,Duck,2536,BDA_18a4_20181106_2_00295_01_04.png,437,294.3125,266.8125,7.0,55.0,59.0,294.3125,36.5
15015,15016,"[145.8125, 17, 62, 60]",3720.0,3,Duck,2536,BDA_18a4_20181106_2_00295_01_04.png,437,176.8125,145.8125,17.0,62.0,60.0,176.8125,47.0
15016,15017,"[-18.1875, 15, 78, 38]",2964.0,3,Duck,2536,BDA_18a4_20181106_2_00295_01_04.png,437,20.8125,-18.1875,15.0,78.0,38.0,20.8125,34.0
15017,15018,"[213.8125, 28, 54, 60]",3240.0,3,Duck,2536,BDA_18a4_20181106_2_00295_01_04.png,437,240.8125,213.8125,28.0,54.0,60.0,240.8125,58.0


In [64]:
#Run DBSCAN
filegroups = df.groupby("filename")
filegroups["cluster_id"]
#filegroups.head()
clusters = filegroups.apply(lambda x: group_DBSCAN(x))


46

In [68]:
clusters

filename
20211201_Atrisco_0465_07_01.png                 [0, 1, 2, 3, 4, 4, 3, 2, 0, 1, 0, 1, -1, 4, 3,...
20220110_Bernardo_0001_01_03.png                [0, 1, 2, 3, -1, 4, 5, 6, 7, 1, 0, 2, 6, 7, 5,...
20220110_Bernardo_0230_05_08.png                [0, -1, -1, 1, 2, 3, 4, 5, 6, 4, 2, 1, 3, 0, 7...
20220110_Bernardo_0999_06_06.png                [0, 10, 1, 2, 3, -1, 4, 5, 12, 6, 7, 8, 9, 0, ...
20220111_Bernardo_0491_03_07.png                [0, 1, 2, 0, 1, 2, 1, 2, 0, 1, 2, -1, 1, 2, 0,...
20220111_LaJoya_1_0024_07_04.png                [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, ...
20220111_LaJoya_1_0024_07_07.png                [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 9, 10, 7, 0...
20220111_LaJoya_2_0196_06_03.png                [0, 1, 2, 5, 3, 4, 4, 3, 5, 1, 2, 0, 2, 1, 0, ...
20220111_LaJoya_2_0196_07_05.png                                  [0, 0, 0, 0, -1, 0, 0, 0, 0, 0]
BDA_12c_20181113_1_00425_01_07.png              [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...
BDA_12c_201

In [78]:
clusters.reset_index("filename")


,filename,0
0,20211201_Atrisco_0465_07_01.png,"[0, 1, 2, 3, 4, 4, 3, 2, 0, 1, 0, 1, -1, 4, 3,..."
1,20220110_Bernardo_0001_01_03.png,"[0, 1, 2, 3, -1, 4, 5, 6, 7, 1, 0, 2, 6, 7, 5,..."
2,20220110_Bernardo_0230_05_08.png,"[0, -1, -1, 1, 2, 3, 4, 5, 6, 4, 2, 1, 3, 0, 7..."
3,20220110_Bernardo_0999_06_06.png,"[0, 10, 1, 2, 3, -1, 4, 5, 12, 6, 7, 8, 9, 0, ..."
4,20220111_Bernardo_0491_03_07.png,"[0, 1, 2, 0, 1, 2, 1, 2, 0, 1, 2, -1, 1, 2, 0,..."
5,20220111_LaJoya_1_0024_07_04.png,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, ..."
6,20220111_LaJoya_1_0024_07_07.png,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 9, 10, 7, 0..."
7,20220111_LaJoya_2_0196_06_03.png,"[0, 1, 2, 5, 3, 4, 4, 3, 5, 1, 2, 0, 2, 1, 0, ..."
8,20220111_LaJoya_2_0196_07_05.png,"[0, 0, 0, 0, -1, 0, 0, 0, 0, 0]"
9,BDA_12c_20181113_1_00425_01_07.png,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


In [ ]:
#DEFINE VISUALIZATION PARAMETERS FOR DBSCAN OUTPUTS
def plot_dbscan(db, X):
    # make a mask for each core point
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True

    plt.figure(figsize=[10, 10])
    unique_labels = set(db.labels_)
    for k in unique_labels:
        kdx = db.labels_ == k
        if k == -1:
            # these are the noise points
            color = 'k'
            plt.scatter(X[kdx, 0], X[kdx, 1], color=color, s=3)
        else:
            color = f'C{k}'
            core_kdx = kdx & core_samples_mask
            non_core_kdx = kdx & ~core_samples_mask
            plt.scatter(X[core_kdx, 0], X[core_kdx, 1], color=color, s=35, edgecolors=color)
            plt.scatter(X[non_core_kdx, 0], X[non_core_kdx, 1], color=color, s=5)

    plt.xlim(0, 20)
    plt.ylim(0, 20)
    plt.xlabel('X')
    plt.ylabel('Y')

In [45]:
#REFINEMENT STEP: DUCK, GOOSE, CRANE

#Dictionary of category values
categories = {1: "Crane", 2: 'Goose', 3:'Duck'}

#Derive median bounding box
refined = filegroups.groupby("cluster_id")['x','y','w','h'].agg(pd.Series.median)
refined.reset_index("cluster_id")
#Make median bounding box into its own column in list form
refined['bbox']= refined.values.tolist()
#Derive plurality vote of class ID
mode_class = filegroups.groupby("cluster_id")["filename",'category_id'].agg(pd.Series.mode)
#Join the refined results together
refined_id = pd.merge(refined, mode_class, left_on="cluster_id", right_on="cluster_id")
#Add back a column with category name to the refined results (optional)
refined_id['category'] = refined_id["category_id"].map(categories)
refined_id = refined_id.drop(columns=['x','y','w','h'])

AttributeError: 'DataFrameGroupBy' object has no attribute 'groupby'

In [ ]:
#REFINEMENT STEP: DGC, SEAGULL

#Dictionary of category values
categories = {1: "Crane", 2: 'Goose', 3:'Duck', 4:'Seagull'}

#Derive median bounding box 
refined = filegroups.groupby("cluster_id")['x','y','w','h'].agg(pd.Series.median)
refined.reset_index("cluster_id")
#Make median bounding box into its own column in list form
refined['bbox']= refined.values.tolist()
#Derive plurality vote of class ID
mode_class = filegroups.groupby("cluster_id")["filename",'category_id'].agg(pd.Series.mode)
#Join the refined results together
refined_id = pd.merge(refined, mode_class, left_on="cluster_id", right_on="cluster_id")
#Add back a column with category name to the refined results (optional)
refined_id['category'] = refined_id["category_id"].map(categories)
refined_id = refined_id.drop(columns=['x','y','w','h'])


In [ ]:
#EXPORT OF REFINED RESULTS FOR USE AS TRAIN/VAL/TEST (not split yet; determine split after getting new data stats)

#add individual annotation ID column
export = refined_id.reset_index
#drop "cluster ID" column
export.drop(columns="cluster_id")
#save to csv
export.to_csv("20230303_consensuslabels.csv")